# PROGETTO ESAME APPRENDIMENTO STATISTICO
#### Emanuele Pinna matr.296455
## Analisi del dataset 'Wine quality'
In questo progetto verrà analizzato il dataset 'winequality-red.csv', contenente campioni del vino rosso 'Vinho Verde' del nord del Portogallo.
L'obbiettivo è quello di classificare i vari campioni nel dataset secondo la qualità del vino in base ai risultati delle prove chimico-fisiche testate nel campione.

<img src='Wine.jpg'/>

##### Per prima cosa si proceda con l'importazione del dataset 'winequality-red.csv', contenente campioni del vino rosso 'Vinho Verde' del nord del Portogallo.
##### L'obbiettivo è quello di modellizzare (e quindi di predire) la qualità del vino rosso basandosi sui risultati delle prove chimico-fisiche.

In [1]:
%matplotlib widget

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display
import sklearn
import FisherDA
#import kfda
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from FisherDA import MultipleFisherDiscriminantAnalysis as FDA
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.decomposition import KernelPCA as KPCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, make_scorer
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
#from kfda import Kfda as KFDA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import seaborn as sn

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

data = pd.read_csv('winequality-red.csv', sep=';')

##### Si vedano ora le prime righe del dataset.

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


<img src='test.png' style='width:1000px'/>

##### Questo dataset è composto da 1599 istanze e 12 features che rappresentano i dati chimico-fisici raccolti dai test.
##### e la dimensione del dataset

In [4]:
data.shape

(1599, 12)

##### Si controllino ora quali in quali colonne delle features si hanno valori mancanti.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


##### Come si può notare dalla tabella sottostante non compaiono missing values, in quanto la riga corrispondente al count presenta solo il valore 1599 (numero delle istanze totali del dataset.

In [6]:
# NON CI SONO MISSING VALUES
data.iloc[:,:-1].describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000


##### vedendo ora la distribuzione delle classi della qualità si nota che le classi sono molto disomogenee e addirittura mancano 5 classi di qualità (0,1,2,9,10)

In [7]:
target_vc = data['quality'].value_counts()
print(target_vc)

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64


In [8]:
S,T = data.shape
qual_freq = pd.concat([data['quality'].value_counts(), data['quality'].value_counts()/S], axis=1)
qual_freq.columns = ['counts', 'freq.']
qual_freq.index.name = 'quality'

qual_freq

,counts,freq.
quality,,
5,681,0.425891
6,638,0.398999
7,199,0.124453
4,53,0.033146
8,18,0.011257
3,10,0.006254


##### di seguito vi è il plot del numero di istanze per ogni qualità

In [9]:
qual_counts = data['quality'].value_counts()

plt.figure()
plt.bar(np.arange(len(qual_counts.values)), qual_counts.values)
plt.xticks(ticks=np.arange(len(qual_counts.values)), 
           labels=qual_counts.index.to_list(),
           rotation=15)
plt.title('Quality DISTRIBUTION')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
#plt.figure(figsize = [10,4])
#color = (0.2, 0.4, 0.6, 0.6)
#target_vc.plot(kind = 'barh', color=color)
#plt.title('QUALITY VALUE COUNTS')
#plt.xlabel('counts')
#plt.ylabel('quality')
#plt.show()

In [11]:
plt.figure(figsize = (12,9))
corrMatrix = data.iloc[:,:-1].corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##### Si osserva che le classi non sono composte dallo stesso numero di campioni
##### Nella creazione di training set e test set per gli algoritmi di classificazione
##### bisognera uniformare il numero di campioni per classe.

##### Si veda ora la distribuzione dei valori all'interno di ciascuna feature.

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1],list(data['quality']),test_size = 0.33, random_state = 7, shuffle = True, stratify = list(data['quality']))

##### dai plot è immediato osservare che non ci sono outliers, tutti i valori per ogni colonna sembrano ben distribuiti

##### Si faccia ora un riscalamento dei dati.

In [52]:
# CONTROLLO RISCALAMENTO DATI
plt.figure(figsize = [15,4])
for col in X_train.columns[0:-1]:
    plt.hist(X_train[col], bins=20, alpha = 0.5)
    plt.xscale('linear')
    plt.title('DISTRIBUTIONS')
    plt.xlabel('values')
    plt.ylabel('samples per bin')
plt.legend(X_train.columns, loc ='right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##### dal precedente grafico si può notare che i valori delle features non sono riscalati in quanto vi è una grande escursione tra i valori delle stesse.

##### Per ovviare questo problema si riscalino tutti i valori delle feature attraverso la funzione di scikit-learn 'MinMaxScaler'.

In [14]:
from sklearn.preprocessing import StandardScaler as SS
scaler = SS().fit
x_scaled = SS().fit_transform(data.iloc[:,0:-1])

pp_data = data.copy()

for col,x in zip(data.columns[0:-1], x_scaled.T):
    pp_data[col] = x

pp_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,5
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385,5
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385,5
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385,6
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,5


In [15]:
# CONTROLLO RISCALAMENTO DATI
plt.figure(figsize = [15,4])
for col in pp_data.columns[0:-1]:
    plt.hist(pp_data[col], bins=20, alpha = 0.5)
    plt.xscale('log')
    plt.title('DISTRIBUTIONS')
    plt.xlabel('values')
    plt.ylabel('samples per bin')
plt.legend(pp_data.columns, loc ='right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plt.figure(figsize = [15,4])
for col in pp_data.columns[0:-1]:
    plt.hist(pp_data[col], bins=20, alpha = 0.5)
    plt.xscale('linear')
    plt.title('DISTRIBUTIONS')
    plt.xlabel('values')
    plt.ylabel('samples per bin')
plt.legend(pp_data.columns, loc ='right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##### Si veda ora, per ogni feature, quale è la distribuzione dei valori di ogni classe. Si può notare che molte feature portano più varianza tra le distribuzioni delle classi rispetto alle altre, questo si vedrà ancora meglio quando si farà la PCA.

In [17]:
# SET DELLE QUALITA' DEI VINI
qualities = set(data['quality'])

# CALCOLO DI MEDIA E VARIANZA CAMPIONARIA
means = []
stds = []
idx = range(len(qualities))

datas = []

for t,i in zip(qualities,idx):
    datas.append(pp_data[pp_data['quality'] == t])
    means.append([])
    stds.append([])
    for col in data.iloc[:,0:-1].columns:
        means[i].append(np.mean(datas[i][col]))
        stds[i].append(np.var(datas[i][col]))
        
means = np.array(means).T
stds = np.array(stds).T


# PLOT DEI RISULTATI OTTENUTI CON IL PREPROCESSING
from scipy.stats import norm

colors = ['b','g','r','c','m','y']
for col,m,v in zip(data.iloc[:,0:-1].columns, means, stds):
    plt.figure(figsize = [10,2])
    plt.title(col)
    for t,i,color in zip(qualities,idx, colors):
        plt.hist(datas[i][col], density=True, color=color, alpha = 0.3)
        x = np.linspace(m[i]-5*v[i],m[i]+5*v[i],100)
        y = norm(m[i],v[i]).pdf(x)
        plt.plot(x,y, color = color)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# SUPERVISED LEARNING
L'apprendimento supervisionato, noto anche come apprendimento automatico supervisionato, è una sottocategoria di apprendimento automatico e intelligenza artificiale .
È definito dall'uso di set di dati etichettati per addestrare algoritmi in grado di classificare i dati o prevedere i risultati in modo accurato.
Quando i dati di input vengono inseriti nel modello, esso aggiusta i propri pesi fino a quando il modello non 'fitta' bene i dati, il che avviene come parte del processo di cross-validation.
# Decision Tree Classifier
In generale i metodi basati sugli alberi sia dei buoni regressori che classificatori.
L'idea principale è quella di dividere lo spazio delle feature $X$ in regioni più piccole per trovare una funzione di predizione per ogni regione.
Poiché l'insieme di regole di divisione utilizzate per segmentare lo
spazio predittore può essere riassunto in un albero, questi tipi di
approcci sono noti come metodi _Decision-Tree_.
##### Vantaggi e svantaggi
<ul>
  <li>I metodi basati sugli alberi sono semplici e di facile interpretazione</li>
  <li>Non sono efficienti come i migliori approcci di supervised learning in termini di accuracy</li>
  <li>Combinando un gran numero di alberi si può spesso ottenere un miglioramento nell'accuracy della predizione a scapito della difficoltà interpretativa</li>
</ul>

##### Processo di Tree-building
<ol>
  <li>Si divide lo spazio dei predittori (l'insieme di tutti i possibili valori per X1, X2,..., Xp) in J regioni distinte non sovrapposte</li>
  <li>Per ogni osservazione che cade nella regione Rj, si fa la stessa predizione, che è semplicemente la media delle variabili response per le osservazioni del training set in Rj</li>
</ol>

In teoria, le regioni potrebbero avere una forma qualsiasi, ma per semplicità di implementazione e di interpretazione si preferisce dividere lo spazio dei predittori in 'scatole alto-dimensionali'.
L'obbiettivo è quello di trovare le scatole R1,...,Rj tale da minimizzare il RSS (Residual Sum of Squares) ovvero la Loss calcolata come somma dei quadrati residua
$$\sum_{j=1}^J\sum_{i\in R_j}(y_i-\hat{y}_{R_j})^2$$
dove con $\hat{y}_{R_j}$ si indica la media dei response per le osservazioni del training set nella j-esima scatola.

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

X = pp_data.iloc[:,:-1]
y = pp_data['quality']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,
                                                   random_state=21,
                                                   stratify=y,shuffle=True)

In [19]:
# TEST DECISION TREE

from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.tree import plot_tree
dtc = DTC(criterion='gini',max_features='log2',
          random_state=7,class_weight='balanced',max_depth=4)
dtc.fit(X_train,y_train)
pred_dtc = dtc.predict(X_test)
print(classification_report(y_test, pred_dtc))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.06      0.36      0.11        11
           5       0.64      0.45      0.53       136
           6       0.47      0.36      0.41       128
           7       0.43      0.23      0.30        40
           8       0.05      0.67      0.10         3

    accuracy                           0.38       320
   macro avg       0.27      0.34      0.24       320
weighted avg       0.51      0.38      0.43       320



In [20]:
plt.figure(figsize=[10,10])
plot_tree(dtc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Random Forest Classifier

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=100, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [22]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='macro')

0.4197355667473053

# UNSUPERVISED LEARNING
L'Unsupervised Learning (o Apprendimento non supervisionato) si riferisce all'uso di algoritmi di intelligenza artificiale per identificare modelli in set di dati contenenti punti dati che non sono né classificati nè etichettati.
In altre parole, l'apprendimento non supervisionato consente al sistema di identificare autonomamente i modelli all'interno del set di dati.
Nell'apprendimento non supervisionato, un sistema di intelligenza artificiale raggrupperà le informazioni non ordinate in base a somiglianze e differenze (tramite le features) anche se non vengono fornite classi.
# Principal Component Analysis


##### Si passi ora all'Analisi delle componenti principali 'PCA'.
##### La Principal Component Analysis (PCA) è uno strumento molto utilizzato per la riduzione lineare della dimensionalità e per l'estrazione delle feature (caratteristiche).
##### Lo scopo della PCA è quello di trovare un sottospazio lineare di dimensionalità fissata inferiore allo spazio delle feature originale, dove le nuove feature abbiano la massima varianza possibile.

#### Principal Component Analysis
La __Principal Component Analysis (PCA)__ è uno strumento molto utilizzato per la riduzione lineare della dimensionalità e per l'estrazione delle feature (caratteristiche).
Lo scopo della PCA è quello di trovare un sottospazio lineare di dimensionalità fissata inferiore allo spazio delle feature originale, dove le nuove feature abbiano la massima varianza possibile.

Si consideri un dataset $\{x_i\}$ dove $i=1,2,...,N$, ed ogni $x_i$ un vettore D-dimensionale.
Ora si vuole proiettare i dati su un sottospazio M-dimensionale, con $M<D$.
Assumendo la proiezione denotata come $y=Ax$, dove $A=[u_1^T,...,u_M^T]$ e $u_k^Tu_k=1$ per $k=1,2,...,M$, si cerca di massimizzare la varianza di $\{y_i\}$, che è la traccia della matrice di covarianza di $\{y_i\}$.
Così, bisogna trovare $$A^*=\arg\max_A tr(S_y)$$ dove $$S_y=\frac{1}{N}\sum_{i=1}^N(y_i-\bar{y})(y_i-\bar{y})^T$$ e $$\bar{y}=\frac{1}{N}\sum_{i=1}^Nx_i.$$
Sia $S_x$ la matrice di covarianza di $\{x_i\}$. Dato che $tr(S_y)=tr(AS_xA^T)$, usando il Moltiplicatore Lagrangiano e prendendo le derivate, si ottiene $$S_xu_k=\lambda_ku_k$$ che significa che $u_k$ è un autovettore di $S_x$. Ora $x_i$ può essere rappresentata come $$x_i=\sum_{k=1}^D(x_i^Tu_k)u_k.$$
$x_i$ può anche essere approssimato da $$\tilde{x}_i=\sum_{i=1}^M(x_i^Tu_k)u_k,$$
dove $u_k$ è l'autovettore di $S_x$ corrispondente al k-esimo autovalore più grande.

##### Si vedano di sequito qualsi sono le componenti principali che spiegano più varianza.

In [23]:
# VOGLIO VEDERE QUALI SONO LE COMPONENTI PRINCIPALI CHE SPIEGANO PIU VARIANZA
from sklearn.decomposition import PCA

color = (0.2, 0.4, 0.6, 0.6)

pca = PCA().fit(pp_data.iloc[:,0:-1],pp_data['quality'])

# scelta componenti principali
plt.figure(figsize = [10,4])
plt.plot(np.arange(1,12),pca.explained_variance_ratio_*100, 'ko')
plt.plot(np.arange(1,12),pca.explained_variance_ratio_*100, 'k')
for i,var in zip(range(1,12), pca.explained_variance_ratio_*100):
    plt.annotate(round(var,2),[i,var])
plt.bar(np.arange(1,12),height = pca.explained_variance_ratio_*100, color = color)
plt.grid()
plt.xlabel('principal components')
plt.xticks(np.arange(1,12))
plt.yticks(np.arange(0,50,5))
plt.ylabel('explained variances')

#plt.semilogy()
plt.title('Wine PCA')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##### Si veda ora come aumenta la varianza spiegata con l'aggiunta delle componenti principali, si noti che il grafico prende la tipica forna a 'gomito' che ottimizza la scelta del numero di componenti principali da considerare.

In [24]:
# controllo sulla distribuzione cumulativa
# scelta componenti principali

plt.figure(figsize = [10,4])
plt.plot(np.arange(1,12),pca.explained_variance_ratio_.cumsum()*100, 'ko')
plt.plot(np.arange(1,12),pca.explained_variance_ratio_.cumsum()*100, 'k')
for i,var in zip(range(1,12), pca.explained_variance_ratio_.cumsum()*100):
    plt.annotate(round(var,2),[i,var])
plt.bar(np.arange(1,12),height = pca.explained_variance_ratio_.cumsum()*100, color = color)
plt.grid()
plt.xlabel('principal components')
plt.xticks(np.arange(1,12))
plt.yticks(np.arange(0,110,10))
plt.ylabel('explained variances')
plt.title('Wine PCA')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
X = pp_data.iloc[:,:-1].values
y = pp_data['quality'].values

In [26]:
pca_wine = PCA()
scaler = StandardScaler()
scaler.fit(X)
Xscaled = scaler.transform(X)
pca_wine.fit(Xscaled)


plt.figure(figsize=(10,8))
plt.plot(100*np.insert(np.cumsum(pca_wine.explained_variance_ratio_), 0, 0))
plt.title('Explained variance of Wine Dataset')
plt.xticks(ticks=np.arange(1, pca_wine.n_features_ + 1), 
           labels=[f'PC{i}' for i in range(1, pca_wine.n_features_ + 1)])
plt.yticks(ticks=np.arange(0, 101, 5))
plt.xlabel('Principal components')
plt.ylabel('Percentage of cumulative explained variance')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##### Qui di seguito è rappresentato invece il grafico bidimensionale e tridimensionale delle classi rispetto alle prime 2 e 3 componenti principali.

In [27]:
#3D plot
data_pca = pca.transform(pp_data.iloc[:,0:-1])

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plt.title('PCA WINE QUALITIES 3D')
for t,c,w in zip(qualities,['purple','c','gold','orange','crimson','mediumblue',
                           'dimgrey'],['quality 3','quality 4','quality 5','quality 6',
                                       'quality 7','quality 8']):
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    ax.scatter(data_pca[data['quality']==t][:,0],
               data_pca[data['quality']==t][:,1],
               data_pca[data['quality']==t][:,2],
               color = c, s = 30, label = w)
ax.legend()
fig.set_size_inches(15,15)
plt.show()

#2D plot
plt.figure(figsize = [8,4])
for t,c,w in zip(qualities,['purple','c','gold','orange','crimson','mediumblue',
                           'dimgrey'],['quality 3','quality 4','quality 5','quality 6',
                                       'quality 7','quality 8']):
    plt.scatter(data_pca[pp_data['quality']==t][:,0],
                data_pca[pp_data['quality']==t][:,1], 
                s = 10, color = c, label = w)
    plt.title('PCA WINE QUALITIES 2D')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
plt.legend()
plt.grid()
plt.show()

<ipython-input-27-4fcf588aa18d>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
pca_wine_m = PCA(0.95)
pca_wine_m.fit(Xscaled)
X_wine_m = pca_wine_m.transform(Xscaled)
num_tosee = 1000
print('Numero PC: {}'.format(pca_wine_m.n_components_))
print('% Varianza Tot. Spiegata: {}'.format(pca_wine_m.explained_variance_ratio_.sum()))


plt.figure(figsize=(8, 6))
plt.scatter(X_wine_m[:num_tosee, 0], X_wine_m[:num_tosee, 1], c=y[:num_tosee], cmap='turbo')
plt.title('WINE - SCORE GRAPH WITH 2 PCA WITH {} POINTS'.format(num_tosee))
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid()
plt.show()

fig_beanscore = plt.figure(figsize=(8, 6))
ax = fig_beanscore.add_subplot(111, projection='3d')
ax.scatter(X_wine_m[:num_tosee, 0], X_wine_m[:num_tosee, 1], X_wine_m[:num_tosee, 2], c=y[:num_tosee], cmap='turbo')
plt.title('WINE - SCORE GRAPH WITH 3 PCA WITH {} POINTS'.format(num_tosee))
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.grid()
plt.show()

Numero PC: 9
% Varianza Tot. Spiegata: 0.9781007746827457


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
plt.figure()
for i in range(pca_wine_m.n_features_):
    plt.plot([0, pca_wine_m.components_[0, i]], [0, pca_wine_m.components_[1, i]], 
             label=pp_data.iloc[:, :-1].columns[i])
plt.scatter(pca_wine_m.components_[0, :], pca_wine_m.components_[1, :], c='k')
plt.legend()
plt.title('WINE - LOADING GRAPH WITH 2 PC')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid()
plt.show()


fig_winescore = plt.figure()
ax = fig_winescore.add_subplot(111, projection='3d')
for i in range(pca_wine_m.n_features_):
    ax.plot([0, pca_wine_m.components_[0, i]], [0, pca_wine_m.components_[1, i]], 
            [0, pca_wine_m.components_[2, i]],
             label=pp_data.iloc[:, :-1].columns[i])
ax.scatter(pca_wine_m.components_[0, :], pca_wine_m.components_[1, :], pca_wine_m.components_[2, :], c='k')
plt.legend(bbox_to_anchor=(1.05, 1), fontsize='xx-small')
plt.title('WINE - LOADING GRAPH WITH 3 PC')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
plt.figure(figsize=(8, 6))
plt.bar(np.arange(pca_wine_m.n_features_), pca_wine_m.components_[0, :])
plt.xticks(ticks=np.arange(pca_wine_m.n_features_), 
           labels=data.iloc[:,:-1].columns.to_list(),
           rotation=30)
plt.title('WINE - PC1')
plt.grid()
plt.show()

plt.figure(figsize=(8, 6))
plt.bar(np.arange(pca_wine_m.n_features_), pca_wine_m.components_[1, :])
plt.xticks(ticks=np.arange(pca_wine_m.n_features_), 
           labels=pp_data.iloc[:,:-1].columns.to_list(),
           rotation=30)
plt.title('WINE - PC2')
plt.grid()
plt.show()

plt.figure(figsize=(8, 6))
plt.bar(np.arange(pca_wine_m.n_features_), pca_wine_m.components_[2, :])
plt.xticks(ticks=np.arange(pca_wine_m.n_features_), 
           labels=pp_data.iloc[:,:-1].columns.to_list(),
           rotation=30)
plt.title('WINE - PC3')
plt.grid()
plt.show()

plt.figure(figsize=(8, 6))
plt.bar(np.arange(pca_wine_m.n_features_), pca_wine_m.components_[3, :])
plt.xticks(ticks=np.arange(pca_wine_m.n_features_), 
           labels=pp_data.iloc[:,:-1].columns.to_list(),
           rotation=30)
plt.title('WINE - PC4')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fisher Discriminant Analysis

#### Fisher Discriminant Analysis
Si supponga di avere un dataset di istanze o di punti $\{(x_i,y_i)\}_{i=1}^n$ con $n$ la grandezza del campione e dimensionalità $x_i\in \mathbb{R}^d$ e $y_i\in\mathbb{R}^l$. Le $\{x_i\}_{i=1}^n$ sono i dati di input del modello e le $\{y_i\}_{i=1}^n$ sono le osservazioni (labels).
Si definisce $\mathbb{R}^{d\times n} \ni X:=[x_1,...,x_n]$ e $\mathbb{R}^{l\times n}\ni Y:=[y_1,...,y_n]$. Di solito, i dati stanno su un sottospazio o una sotto-collezione.
Qui, si considera il caso in cui le osservazioni $\{y_i\}_{i=1}^n$ vengano da un insieme discreto in modo tale che l'obiettivo sia la _classificazione_.
Supponendo che il dataset consista di $c$ classi, $\{x_i^{(1)}\}_{i=1}^{n_1},...,\{x_i^{(c)}\}_{i=1}^{n_c}$ dove $n_j$ denota la cardinalità del campione della j-esima classe.
Si vuole pertanto trovare un sottospazio (o sotto-collezione) che separi le classi il più possibile mentre i dati diventino il più sparsi possibile.
La __Fisher Discriminant Analysis (FDA)__ persegue questo obiettivo cercando di massimizzare la distanza tra le classi proiettate e contemporaneamente minimizzare la varianza all'interno delle singole classi.
La varianza delle classi viene rappresentata dalla cosiddetta _Between Scatter Matrix_ $S_B$

__Sottospazio Multidimensionale__
Nel caso in cui il sottospazio di Fisher è lo span di più direzioni di Fisher, $\{u_j\}_{i=1}^p$ dove $u_j \in \mathbb{R}^p$, il $d_B$ e $d_W$ sono definiti come:
$$\mathbb{R} \ni d_B := tr(U^TS_BU),$$
$$\mathbb{R} \ni d_W := tr(U^TS_WU),$$
dove $\mathbb{R}^{d\times p}\ni U=[u_1,...,u_p]$. In questo caso, massimizzare il _criterio di Fisher_ significa:
$\underset{U}{mazimize}\;f(U):= \frac{d_B(U)}{d_W(U)}=\frac{tr(U^TS_BU)}{tr(U^TS_WU)}$.
Il _criterio di Fisher_ $f(U)$ è un _quoziente generalizzato di Rayleigh-Ritz_.
Pertanto l'ottimizzazione dell'equazione di sopra è equivalente a:
$$\underset{U}{mazimize}\;tr(U^TS_BU)$$
$$tale\;che\;U^TS_WU=I$$
Il _Lagrangiano_ è:
$$\mathcal{L}=tr(U^TS_BU)-tr(\Lambda^T(U^TS_WU-I))$$
dove $\Lambda\in\mathbb{R}^{d\times d}$ è una matrice diagonale le cui entrate diagonali sono i _moltiplicatori di Lagrange_.
Eguagliando le derivate di $\mathcal{L}$ a zero si ottiene:
$$\mathcal{R}^{d\times d}\ni\frac{\partial \mathcal{L}}{\partial U}=2S_WU\Lambda \overset{set}{=}0\implies 2S_BU=2S_WU\Lambda\implies S_BU=S_WU\Lambda,$$
che è un problema agli autovalori generalizzati $(S_B,S_W)$.
Le colonne di $U$ sono gli autovettori ordinati dall'autovalore più grande al più piccolo (perchè in questo caso l'ottimizzazione è la massimizzazione) e le entrate diagonali di $\Lambda$ sono gli autovalori corrispondenti. Le colonne di $U$ vengono anche chiamate le _direzioni di Fisher_ o gli _assi di Fisher_.
Una possibile soluzione al problema agli autovalori generalizzati $(S_B,S_W)$ è:
$$S_BU=S_wU\Lambda\implies S_W^{-1}S_BU=U\Lambda\implies U=eig((S_W+\epsilon I)^{-1}S_B),$$
Un altro modo per risolvere l'ottimizzazione sarebbe prendere le derivate dal _criterio di Fisher_:
$$\mathbb{R}^{d\times p}\ni\frac{\partial f(U)}{\partial U}=\frac{1}{(tr(U^TS_WU))^2}\times [tr(U^TS_WU)(2S_BU)-tr(U^TS_BU)(2S_WU)]\overset{set}{=}0\overset{(a)}{\implies}S_BU=\frac{tr(U^TS_BU}{tr(U^TS_WU)}S_WU,$$
dove $(a)$ vale in quanto $tr(U^TS_WU)$ è uno scalare.
L'equazione sopra è un problema agli autovalori generalizzati $(S_B,S_W)$ con colonne di $U$ come autovettori e $(u_j^TS_Bu_j)/(u_j^TS_Wu_j)$ come il j-esimo autovalore (in quanto l'ottimizzazione è la massimizzazione).
Di nuovo, un altro modo di trovare le direzioni FDA è di considerare un'altra versione del criterio di Fisher.
Il _criterio di Fisher_ diventa
$$f(U)=\frac{tr(U^T(S_T-S_W)U)}{tr(U^TS_WU)}=\frac{tr(U^TS_TU)}{tr(U^TS_WU)}-1.$$
Il $-1$ è una costante ed viene persa nell'ottimizzazione, pertanto:
$$\underset{U}{mazimize}\;tr(U^TS_BU)$$
$$tale\;che\;U^TS_WU=I$$
le cui soluzioni sono similmente ottenute come:
$$S_TU=S_WU\Lambda,$$
che è un problema agli autovalori generalizzati $(S_T,S_W)$.

In [31]:
fda_wine2 = FDA(n_dimensions=2)
fda_wine3 = FDA(n_dimensions=3)
fda_wine2.fit(X,y)
fda_wine3.fit(X,y)
Z2m = fda_wine2.transform(X)
Z3m = fda_wine3.transform(X)

plt.figure(figsize=(8, 6))
plt.scatter(Z2m[:num_tosee, 0], Z2m[:num_tosee, 1], c=y[:num_tosee], cmap='turbo')
plt.title('WINE - FDA GRAPH WITH 2 DIM WITH {} POINTS'.format(num_tosee))
plt.grid()
plt.show()

fig_winescore = plt.figure(figsize=(8, 6))
ax = fig_winescore.add_subplot(111, projection='3d')
ax.scatter(Z3m[:num_tosee, 0], Z3m[:num_tosee, 1], Z3m[:num_tosee, 2], c=y[:num_tosee], cmap='turbo')
plt.title('WINE - FDA GRAPH WITH 3 DIM WITH {} POINTS'.format(num_tosee))
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

c:\users\emanu\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_asarray.py:171: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order, subok=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

c:\users\emanu\appdata\local\programs\python\python38\lib\site-packages\numpy\core\_asarray.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [32]:
# Plot a confronto

class_colors = [plt.cm.tab10.colors[c] for c in y[:num_tosee]]

# Plot per proiezione in R^2
fig2, axs2 = plt.subplots(1, 2, figsize=(8, 3))
axs2[0].scatter(Z2m[:num_tosee, 0], Z2m[:num_tosee, 1], c=class_colors, alpha=0.15, cmap='turbo')
axs2[0].set_title('FDA')
axs2[1].scatter(X_wine_m[:num_tosee,0], X_wine_m[:num_tosee,1], c=class_colors, alpha=0.15, cmap='turbo')
axs2[1].set_title('PCA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'PCA')

In [33]:
# Plot a confronto

# Plot per proiezione in R^3
fig_Z3m = plt.figure(figsize=(10, 8))
ax_Z3m = fig_Z3m.add_subplot(111, projection='3d')
ax_Z3m.scatter(Z3m[:num_tosee, 0], Z3m[:num_tosee, 1], Z3m[:num_tosee, 2], c=y[:num_tosee], alpha=0.15, cmap='turbo')
plt.title('FDA')

fig_Z3p = plt.figure(figsize=(10, 8))
ax_Z3p = fig_Z3p.add_subplot(111, projection='3d')
ax_Z3p.scatter(X_wine_m[:num_tosee, 0], X_wine_m[:num_tosee, 1], X_wine_m[:num_tosee, 2], c=y[:num_tosee], alpha=0.15, cmap='turbo')
plt.title('PCA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.92, 'PCA')

In [34]:
random_seed = 20210528
kpca_wine = KPCA(kernel='poly', degree=2, coef0 = 4, gamma=0.4, random_state=random_seed, fit_inverse_transform=True)
#expld_variance_global = np.insert(np.cumsum(kpca_iris.explained_variance_ratio_), 0, 0)
Xkpca = kpca_wine.fit_transform(Xscaled[:num_tosee, :])

plt.figure(figsize=(8, 6))
plt.scatter(Xkpca[:num_tosee, 0], Xkpca[:num_tosee, 1], c=y[:num_tosee], cmap='turbo')
plt.title('WINE - KPCA SCORE GRAPH WITH 2 PC')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid()
plt.show()


fig_winescoreK = plt.figure()
ax = fig_winescoreK.add_subplot(111, projection='3d')
ax.scatter(Xkpca[:num_tosee, 0], Xkpca[:num_tosee, 1], Xkpca[:num_tosee, 2], c=y[:num_tosee], cmap='turbo')
plt.title('WINE - KPCA SCORE GRAPH WITH 3 PC')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
plt.figure(figsize=(10,8))
#plt.plot(np.insert(100*np.cumsum(kpca_bean.lambdas_)/np.sum(kpca_bean.lambdas_), 0, 0))
plt.plot(np.insert(100*np.cumsum(kpca_wine.lambdas_)/np.sum(kpca_wine.lambdas_), 0, 0))
plt.title('Explained variance of Wine Dataset')
plt.xticks(ticks=np.arange(1, np.size(kpca_wine.lambdas_) + 1,10), 
           labels=[f'PC{i}' for i in range(1, np.size(kpca_wine.lambdas_) + 1,10)])
plt.yticks(ticks=np.arange(0, 101, 5))
plt.xlabel('Principal components')
plt.ylabel('Percentage of cumulative explained variance')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
def my_scorer(estimator, X, y=None):
    X_reduced = estimator.transform(X)
    X_preimage = estimator.inverse_transform(X_reduced)
    return -1 * mean_squared_error(X, X_preimage)

n_features = data.shape[1]
ker_list = ['poly','linear','rbf', 'cosine']
gamma_list = [1/(i*n_features) for i in np.arange(0.5, 1.5, 0.25)]
deg_list = [i for i in np.arange(1,4,1)]
coef_list = [2**i for i in range(-2,3)]

hparameters = {'kernel':ker_list, 'coef0':coef_list, 'gamma':gamma_list, 'degree':deg_list}
kpca = KPCA(n_components=3, fit_inverse_transform=True)

wine_kpca_GS = GridSearchCV(estimator=kpca, 
                      param_grid=hparameters, 
                      scoring=my_scorer,
                      return_train_score=True)
wine_kpca_GS.fit(Xscaled[:num_tosee, :])
df_results = pd.DataFrame(wine_kpca_GS.cv_results_)
#indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 16]
#df = df_results.iloc[:, indices]
display(df_results.sort_values(['rank_test_score'], ascending=True))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_coef0,param_degree,param_gamma,param_kernel,params,split0_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
172,0.054158,0.003379,0.004559,0.000798,2,2,0.066667,poly,"{'coef0': 2, 'degree': 2, 'gamma': 0.066666666...",-0.817437,...,-0.800432,0.050321,1,-3.970366e-27,-5.082449e-27,-5.438893e-27,-4.773527e-27,-4.327412e-27,-4.718530e-27,5.228972e-28
216,0.049181,0.003380,0.004148,0.000400,4,2,0.083333,poly,"{'coef0': 4, 'degree': 2, 'gamma': 0.083333333...",-0.970466,...,-0.817475,0.087986,2,-5.464408e-26,-7.548223e-26,-9.043889e-26,-8.599250e-26,-6.707588e-26,-7.472672e-26,1.293220e-26
220,0.049599,0.004135,0.003707,0.000239,4,2,0.066667,poly,"{'coef0': 4, 'degree': 2, 'gamma': 0.066666666...",-0.987052,...,-0.818788,0.095433,3,-5.007002e-26,-7.772818e-26,-8.098842e-26,-7.208128e-26,-6.347000e-26,-6.886758e-26,1.111967e-26
168,0.057281,0.004680,0.003962,0.000631,2,2,0.083333,poly,"{'coef0': 2, 'degree': 2, 'gamma': 0.083333333...",-0.896266,...,-0.818876,0.059599,4,-4.022218e-27,-5.062550e-27,-6.705227e-27,-5.020689e-27,-4.741119e-27,-5.110361e-27,8.802637e-28
212,0.051194,0.002322,0.004588,0.000507,4,2,0.111111,poly,"{'coef0': 4, 'degree': 2, 'gamma': 0.111111111...",-1.012505,...,-0.832083,0.099238,5,-7.089627e-26,-8.974315e-26,-1.139157e-25,-1.029510e-25,-9.839870e-26,-9.518097e-26,1.442937e-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,0.089659,0.003536,0.013793,0.000785,4,3,0.166667,poly,"{'coef0': 4, 'degree': 3, 'gamma': 0.166666666...",-1234.379532,...,-251.781660,491.367252,236,-1.888085e-15,-1.559502e-13,-6.732692e-13,-4.953957e-13,-6.287917e-13,-3.910590e-13,2.659984e-13
176,0.098474,0.006432,0.015191,0.001537,2,3,0.166667,poly,"{'coef0': 2, 'degree': 3, 'gamma': 0.166666666...",-1934.959244,...,-401.465315,767.194737,237,-6.352877e-16,-7.265932e-15,-2.867680e-14,-1.637348e-14,-6.106996e-14,-2.280429e-14,2.132139e-14
128,0.101254,0.003551,0.016357,0.001194,1,3,0.166667,poly,"{'coef0': 1, 'degree': 3, 'gamma': 0.166666666...",-13238.064983,...,-2665.527533,5286.368696,238,-2.140480e-17,-2.822943e-14,-2.338502e-15,-1.738997e-14,-8.816886e-15,-1.135924e-14,1.036684e-14
32,0.087679,0.003383,0.014490,0.000662,0.25,3,0.166667,poly,"{'coef0': 0.25, 'degree': 3, 'gamma': 0.166666...",-20428.482781,...,-4098.544664,8165.000424,239,-7.487545e-18,-2.562352e-15,-2.180329e-15,-3.544317e-15,-2.793784e-15,-2.217654e-15,1.191386e-15


In [37]:
wine_kpca_GS.best_estimator_.fit(Xscaled[:num_tosee, :])
X_wine_K = wine_kpca_GS.transform(Xscaled[:num_tosee, :])

plt.figure(figsize=(8, 6))
plt.scatter(X_wine_K[:num_tosee, 0], X_wine_K[:num_tosee, 1], c=y[:num_tosee], cmap='turbo')
plt.title('WINE - KPCA SCORE GRAPH WITH 2 PCA WITH {} POINTS'.format(num_tosee))
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid()
plt.show()

fig_winescore = plt.figure(figsize=(8, 6))
ax = fig_winescore.add_subplot(111, projection='3d')
ax.scatter(X_wine_K[:num_tosee, 0], X_wine_K[:num_tosee, 1], X_wine_K[:num_tosee, 2], c=y[:num_tosee], cmap='turbo')
plt.title('WINE - KPCA SCORE GRAPH WITH 3 PCA WITH {} POINTS'.format(num_tosee))
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Appendice sugli RKHS

## 1. REPRODUCING KERNEL HILBERT SPACES
### 1.1 _Definizione di funzionale di valutazione di Dirac_
Sia $ \mathbb{H} $ lo spazio $ \mathbb{L}_2 (X) $ di funzioni da $X$ in $\mathbb{R}$ per qualche insieme non vuoto $X$. Si definisce il __funzionale di valutazione di Dirac in $x$__ il funzionale $\delta_x$ tale che $$ \delta_x : \mathbb{H} \to \mathbb{R}\quad \text{tale che}\quad\delta_x (f) = f(x) $$
$\delta_x$ è continuo se esiste una costante $M>0$, $M \in \mathbb{R}$ tale che $$ \lvert\lvert{\delta_x (f)\rvert\rvert}_{\mathbb{L}_2} = \lvert{f(x)\rvert} \leq M \lvert\lvert{f\rvert\rvert}_{\mathbb{L}_2} \quad \forall f \in \mathbb{H}, \quad \forall x \in \mathbb{R}^n $$

### 1.2 _Definizione di RKHS_
Un __RKHS__ (Reproducing Kernel Hilbert Space) è uno spazio di Hilbert in cui ogni funzionale di valutazione di Dirac è continuo.

### 1.3 _Teorema di rappresentazione di Riesz_
Sia $\mathbb{H}$ uno spazio di Hilbert. Allora per ogni funzionale lineare e continuo $\Phi$ su $\mathbb{H}$, esiste ed è unico un elemento $u$ di $\mathbb{H}$ tale che $$ \Phi (f) = \langle f, u \rangle_{\mathbb{H}} $$

Nel caso dei funzionali di valutazione di Dirac, se siamo in un RKHS allora per ogni $\delta_x$ esiste ed è unico $k_x \in \mathbb{H}$ tale che $$ \delta_x (f) = f(x) = \langle f, k_x \rangle_{\mathbb{H}} $$
$k_x$ viene detto __representer in $x$__.

### 1.4 _Definizione di Reproducing Kernel_
Sia $\mathbb{H}$ uno spazio di Hilbert di funzioni da $X$ in $\mathbb{R}$ con $X$ insieme non vuoto. Una funzione $\mathcal{K} : X \times X \to \mathbb{R}$ si dice __Reproducing Kernel__ di $\mathbb{H}$ se soddisfa:
* $\forall x \in X \quad k_x = \mathcal{K}(\cdot,x) \in \mathbb{H}$, $k_x$ si dice __representer in $x$__
* $\mathcal{K}(x,x) < \infty \quad \forall x \in X$
* $\forall x \in X, \forall f \in \mathbb{H} \quad \langle f, \mathcal{K}(\cdot,x) \rangle = f(x)$ (__Reproducing Property__)

Osserviamo che $k_x \in \mathbb{H}$ è una funzione da $X$ in $\mathbb{R}$ tale che $k_x (y) = \mathcal{K}(x,y)$
In particolare, per ogni $x,y \in X$ $$ \langle k_x, k_y \rangle_{\mathbb{H}} = \langle \mathcal{K}(\cdot,x), \mathcal{K}(\cdot,y) \rangle_{\mathbb{H}} = \mathcal{K}(x,y) = \mathcal{K}(y,x) $$ dove $k_x, k_y$ sono rispettivamente i representer di $\delta_x, \delta_y$. Segue chiaramente che $\mathcal{K}$ è simmetrico e definito positivo, perche discende dal prodotto scalare su $\mathbb{H}$.

Chiaramente se $k_x \in \mathbb{H} \;\; \forall x \in X$, allora $\textbf{span}\{ k_x : x \in X \} \subseteq \mathbb{H}$, quindi se $f = \sum_{j=1}^N \alpha_j \mathcal{K}(\cdot,x_j)$ con $x_j \in X$, allora $$ \vert\vert f \vert\vert_{\mathbb{H}}^2 = \sum_{j=1}^N \sum_{i=1}^N \alpha_j \alpha_i \langle \mathcal{K}(\cdot, x_j),\mathcal{K}(\cdot, x_i) \rangle_{\mathbb{H}} = \sum_{j=1}^N\sum_{i=1}^N \alpha_j \alpha_i \mathcal{K}(x_j, x_i) $$

### 1.5 _Teorema di unicità_
Se esiste, il reproducing kernel $\mathcal{K}$ per uno spazio di Hilbert $\mathbb{H}$ è unico.
##### _Dimostrazione_
Assumiamo che $\mathbb{H}$ abbia due Reproducing Kernel $\mathcal{K}_1$ e $\mathcal{K}_2$. Allora $$ \langle f, \mathcal{K}_1 (\cdot, x) - \mathcal{K}_2 (\cdot, x) \rangle = f(x) - f(x) = 0 \quad \forall f \in \mathbb{H},\; \forall x \in X $$
In particolare, se prendiamo $f = \mathcal{K}_1 (\cdot, x) - \mathcal{K}_2 (\cdot, x)$ otteniamo $$ \vert\vert \mathcal{K}_1 (\cdot, x) - \mathcal{K}_2 (\cdot, x) \vert\vert_{\mathbb{H}}^2 = 0 \;\; \forall x \in X \implies \mathcal{K}_1 = \mathcal{K}_2 $$

### 1.6 _Equivalenza tra Reproducing Kernel e $\delta_x$ limitato_
Sia $\mathbb{H}$ uno spazio di Hilbert di funzioni $f : X \to\mathbb{R}$. Allora gli operatori di valutazione $\delta_x$ sono funzionali limitati e continui $\iff$ $\mathbb{H}$ ha un Reproducing Kernel $\mathcal{K}$
##### _dimostrazione_
$( \Longleftarrow )$ Poiché $\mathbb{H}$ è uno spazio di Hilbert con Reproducing Kernel $\mathcal{K}$, allora $$ | \delta_x f | = | f(x) | = | \langle f, \mathcal{K} (\cdot, x) \rangle_{\mathbb{H}} | \leq || \mathcal{K}(\cdot, x ) ||_{\mathbb{H}} || f ||_{\mathbb{H}} = \langle \mathcal{K}(\cdot, x), \mathcal{K}(\cdot, x) \rangle_{\mathbb{H}}^{\frac{1}{2}} || f ||_{\mathbb{H}} = \mathcal{K}(x,x)^{\frac{1}{2}} || f ||_{\mathbb{H}} $$
$ \implies $ $\delta_x : \mathbb{H} \to \mathbb{R}$ è un operatore lineare e limitato.

$ ( \Longrightarrow ) $ Supponiamo che $\delta_x : \mathbb{H} \to \mathbb{R}$ sia un funzionale lineare e limitato.
Allora per il teorema di rappresentazione di Riesz $\implies$ $\exists f_{\delta_x} \in \mathbb{H}$ tale che $\delta_x f = \langle f, f_{\delta_x} \rangle_{\mathbb{H}}\;\; \forall f \in \mathbb{H} $. Definiamo $\mathcal{K} (x', x) = f_{\delta_x} (x') \quad \forall x, x' \in X$. Allora $$ \mathcal{K} (\cdot, x) = f_{\delta_x} \;\; \text{e}\;\; \langle f, \mathcal{K} (\cdot, x) \rangle_{\mathbb{H}} = \delta_x f = f(x) $$
$\implies$ $\mathcal{K}$ è un Reproducing Kernel per $\mathbb{H}$

### 1.7 _Definizione di Kernel_
Una funzione $\mathcal{K} : X \times X \to \mathbb{R} $ si dice __Kernel__ su un insieme non vuoto $X$ se esiste uno spazio di Hilbert $\mathbb{H}$ (non necessariamente RKHS) e una mappa $\Phi : X \to \mathbb{H}$, tale che $$ \mathcal{K}(x,y) = \langle \Phi(x), \Phi(y) \rangle_{\mathbb{H}} $$
$\Phi$ è chiamata __Feature Map__, e $\mathbb{H}$ è chiamato __Feature Space__

Si dimostra che ogni Reproducing Kernel è anche un Kernel. 
Basta prendere come $\Phi$ la funzione che manda $ x \mapsto \mathcal{K}(\cdot,x) = k_x$, con $k_x \in \mathbb{H}$ e quindi $\mathcal{K}(x,y) = \langle \mathcal{K},\mathcal{K}(\cdot, y) \rangle_{\mathbb{H}}$. Allora il RKHS $\mathbb{H}$ è il Feature Space.
__N.B.__ Non vale il viceversa, in quanto verrebbe meno l'unicità.

Un Kernel $\mathcal{K}$ può essere visto come una matrice di infinite dimensioni, ma nonostante questo esistono due diverse definizioni di Kernel definito positivo, equivalenti nel caso di Kernel continui.

#### 1.7.1 _Kernel definito positivo_
Un Kernel simmetrico $\mathcal{K} : X \times X \to \mathbb{R}$ si dice __(semi-)definito positivo__ su $X$ se la sua matrice di kernel associata $(\mathcal{K})_{i,j} = \mathcal{K}(x_i, x_j)_{i,j=1}^N$ è (semi-)definita positiva $\forall N \in \mathbb{N}$ e per ogni insieme di punti distinti $\{ x_1, \ldots, x_N \} \subset X$.

#### 1.7.2 _Kernel integralmente definito positivo_
Un Kernel simmetrico $\mathcal{K} : X \times X \to \mathbb{R}$ si dice __semi-definito positivo in $\mathbb{L}_2$__ se per ogni funzione $f \in \mathbb{L}_2$ si ha che $$ \int_X \int_X f(x) \mathcal{K}(x,x') f(x') dx dx' \geq 0 $$
ed si dice __definito positivo in $\mathbb{L}_2$__ se l'integrale è uguale a $0$ $\iff$ $f \equiv 0$ quasi ovunque.

#### 1.8 _Teorema_
Ogni Kernel è una funzione semi-definita positiva.

##### _Dimostrazione_
Siano $\{ x_1, \ldots, x_N \} \subset X$ distinti e $c \in \mathbb{R}^N$. Abbiamo che 
$$ \sum_{i=1}^N \sum_{j=1}^N c_i c_j \mathcal{K} (x_i, x_j) =\sum_{i=1}^N \sum_{j=1}^N c_i c_j \langle \phi (x_i), \phi (x_j) \rangle_{\mathbb{H}} =\bigg\langle \sum_{i=1}^N c_i \phi (x_i), \sum_{j=1}^N c_j \phi (x_j) \bigg\rangle_{\mathbb{H}} = \bigg|\bigg| \sum_{i=1}^N c_i \phi (x_i) \bigg|\bigg|_{\mathbb{H}}^2 \geq 0$$
Reproducing Kernel $\implies$ Kernel $\implies$ funzione semi-definita positiva

Un primo modo per costruire un RKHS a partire da un Kernel e dato dal seguente risultato
### 1.9 _Teorema di Moore-Aronszajn_
Sia $\mathcal{K} : X \times X \to \mathbb{R}$ un kernel definito positivo. Allora esiste ed è unico (a meno di isomorfismi) un RKHS $\mathbb{H}_{\mathcal{K}} \in \mathbb{R}^X$ con Reproducing Kernel $\mathcal{K}$.

##### _Dimostrazione Teorema_
Per ipotesi $\mathcal{K}$ è un Kernel definito positivo. Definiamo $\mathbb{H}_0 = \textbf{span} \{ k_x : x \in X \}$, per cui ogni elemento di $\mathbb{H}_0$ può essere scritto come $\sum_i \alpha_i k_{x_i} = \sum_i \alpha_i \mathcal{K}(\cdot, x_i) $, e ci definiamo una forma bilineare $$ \langle f_1, f_2 \rangle_{\mathcal{K}} = \bigg\langle \sum_{i=1}^N \alpha_i \mathcal{K}(\cdot, x_i), \sum_{j=1}^N \beta_j \mathcal{K}(\cdot, y_j) \bigg\rangle_{\mathcal{K}} =\sum_{i=1}^N \sum_{j=1}^N \alpha_i \beta_j \mathcal{K} (x_i, y_j) $$
##### 1.9.1 _Lemma_
Se $\mathcal{K} : X \times X \to \mathbb{R}$ è un Kernel simmetrico definito positivo $\implies$ $\langle \cdot, \cdot \rangle_{\mathcal{K}}$ definisce un prodotto interno su $\mathbb{H}_0$. Inoltre $\mathbb{H}_0$ è uno spazio pre-hilbertiano con Reproducing Kernel $\mathcal{K}$.
##### _Dimostrazione Lemma_
Ovviamente $\langle \cdot, \cdot \rangle_{\mathcal{K}}$ è bilineare e simmetrico. Inoltre, se consideriamo una qualsiasi funzione $f = \sum_{j=1}^N \alpha_j \mathcal{K}(\cdot, x_j) \not= 0$ in $\mathbb{H}_0$, troviamo che $$ \langle f, f \rangle_{\mathcal{K}} = \sum_{j = 1}^N \sum_{k = 1}^N \alpha_j \alpha_k \mathcal{K} (x_j, x_k) > 0 $$
perché per ipotesi $\mathcal{K}$ è definito positivo. Infine otteniamo che $$ \langle f, \mathcal{K}(\cdot, y) \rangle_{\mathcal{K}} = \sum_{j=1}^N \alpha_j \mathcal{K} (x_j, y) = f(y) $$
$\implies$ $\mathbb{H}_0$ è uno spazio pre-hilbertiano con Reproducing Kernel $\mathcal{K}$.

Notiamo però che $\mathbb{H}_0$ non è necessariamente completo, per cui cerchiamo di farne il completamento. Definiamo quindi $\mathbb{H}_{\mathcal{K}}$ come lo spazio di funzioni $f \in \mathbb{R}^X$ per esiste una sequenza $\{ f_n \}$ di Cauchy in $\mathbb{H}_0$. Fissiamo $x \in X$ e consideriamo $$ | f_n (x) - f_m (x) | = | \langle \mathcal{K}(x,\cdot), f_n - f_m \rangle | \leq || \mathcal{K} (x,\cdot) ||_{\mathbb{H}_0} || f_n - f_m ||_{\mathbb{H}_0} = \langle \mathcal{K}(x,\cdot), \mathcal{K}(x,\cdot) \rangle^{\frac{1}{2}}_{\mathbb{H}_0} || f_n - f_m ||_{\mathbb{H}_0} = \mathcal{K}(x,x)^{\frac{1}{2}} || f_n - f_m ||_{\mathbb{H}_0} $$ 
Per definizione sappiamo che $\mathcal{K}(x,x) < \infty \;\; \forall x \in X$, da cui segue che $\{ f_n (x) \}$ è una successione limitata di Cauchy in $\mathbb{R}$, che è completo $\implies$ esiste $f(x) = \lim f_n (x)$. Allora aggiungiamo a $\mathbb{H}_0$ tutte le funzioni così definite per ottenere $\mathbb{H}_{\mathcal{K}}$, per cui infine $\forall x \in X$ abbiamo che 
$$ f(x) = \lim_{n \to \infty} f_n (x) = \lim_{n \to \infty} \langle f_n, \mathcal{K}(\cdot,x) \rangle_{\mathbb{H}_0} = \langle f, \mathcal{K}(\cdot,x) \rangle_{\mathbb{H}_{\mathcal{K}}} $$ per cui $\mathcal{K}$ e' il Reproducing Kernel di $\mathbb{H}_{\mathcal{K}}$.

Un'altra costruzione di RKHS a partire da un Kernel viene data in termini di autofunzioni associate all'operatore con nucleo integrale definito da $$ T_{\mathcal{K}}(q)(x) = \int_X \mathcal{K}(x,y) q(y) dy \quad q \in \mathbb{L}_2 (X), \; x \in X $$
Affinche questo operatore sia ben definito, e necessario che la norma in $\mathbb{L}_2 (X)$ sia finita $$ \int \int \mathcal{K}^2 (x,x') dx dx' < \infty \implies \mathcal{K}(\cdot,x) \in \mathbb{L}_2 (X) $$
Questa proprieta si dice __traccia finita__.
### 1.10 _Teorema di Mercer_
Sia $\mathcal{K}$ un Kernel definito positivo, continuo e con traccia finita. Sia $T_{\mathcal{K}}$ l'operatore con nucleo integrale definito sopra, che sappiamo essere ben definito. Allora esiste una sequenza infinita di autofunzioni $\{ \phi_i \}_{i =0}^{\infty} $ e di autovalori $\{ \lambda_i \}_{i=0}^{\infty}$ di $T_{\mathcal{K}}$, con $\lambda_1 \geq \lambda_2 \geq \ldots$ tale che $\mathcal{K}$ puo essere scritto come $$ \mathcal{K} (x,x') = \sum_{i = 0}^{\infty} \lambda_i \phi_i (x) \phi_i (x') $$

Questo risultato permette di costruire un RKHS $\mathbb{H}_{\mathcal{K}}$ in questo modo: $$ \mathbb{H}_{\mathcal{K}} := \bigg\{ f : f = \sum_{i = 0}^{\infty} c_i \phi_i \bigg\} $$
Chiaramente $f \in \mathbb{L}_2$, per cui i coefficienti saranno dati da $$ f_i = \langle f, \phi_i \rangle_{\mathbb{L}_2} = \int f(x) \phi_i (x) dx $$
E' un risultato base dell'analisi di Fourier che tale rappresentazione esiste ed e' unica.

Dato cio', possiamo ora definire il prodotto interno su $\mathbb {H}_{\mathcal{K}}$:
$$\langle f,g \rangle_{\mathbb{H}_{\mathcal{K}}} = \sum_{i=0}^{\infty} \frac{f_i g_i}{\lambda_i}$$
dove abbiamo usato la $\mathbb{H}_{\mathcal{K}}$-ortogonalita' $\langle \phi_j, \phi_k \rangle_{\mathbb{H}_{\mathcal{K}}} = \cfrac{\delta_{j,k}}{\sqrt{\lambda_j}\sqrt{\lambda_k}}$ delle autofunzioni.

__Nota:__ $\mathcal{K}$ e' il reproducing kernel di $\mathbb{H}_{\mathcal{K}}$ dato che l'espansione in autofunzioni di $\mathcal{K}$, data dal _Teorema di Mercer_, e l'ortogonalita' delle autofunzioni implicano che $$\langle f,\mathcal{K}(\cdot,x)\rangle_{\mathbb{H}_{\mathcal{K}}} = \bigg\langle\sum_{j=0}^\infty c_j\phi_j, \sum_{j=0}^\infty \lambda_i \phi_i \phi_i(x)\bigg\rangle_{\mathbb{H}_{\mathcal{K}}} = \sum_{i=0}^\infty \cfrac{c_i\lambda_i\phi_i(x)}{\lambda_i} = \sum_{i=0}^\infty c_i\phi_i(x) = f(x)$$

### 1.11 _Kernel trick_
Dato un kernel $\mathcal{K}$ esiste una feature map associata $\Phi : X\to\mathbb{H}$ tale che
$$\mathcal{K}(x,x') = \langle \Phi(x),\Phi(x')\rangle_{\mathbb{H}}$$
per esempio dato un kernel $\mathcal{K}$ esiste una funzione $\Phi$ tale che la valutazione del kernel nei punti $x$ e $x'$ e' equivalente a prendere il profotto scalare tra $\Phi(x)$ $\Phi(x')$ in alcuni (forse sconosciuti) spazi di Hilbert.

Questo ci da' la possibilita' di utilizzare il _kernel trick_, nel quale trasformiamo gli input in $\mathbb{H}$ usando $\Phi$ e quindi prendiamo il prodotto scalare come prima.

Abbiamo visto come costruire un _RKHS_ $\mathbb{H}$ partendo da un kernel definito positivo e che $\mathbb{H}_{\mathcal{K}}$ e' unico a meno di isomorfismi.

Questo significa che $\Phi$ non e' unico in modo assoluto, ma lo e' preso $\mathbb{H}$.